# Random forest inference

In [5]:
import joblib
import numpy as np
import warnings

from sklearn.exceptions import InconsistentVersionWarning

warnings.filterwarnings("ignore", category=InconsistentVersionWarning)

In [3]:
embedded_string = """
 0.09815001  0.01886502  0.00166107 -0.07299203  0.08320449 -0.0100925
  0.00935933  0.06141649  0.00717163 -0.02368737 -0.07750969 -0.02453464
  0.04996623  0.02022458  0.02726015 -0.01597108 -0.01577094  0.04192893
  0.06735542 -0.01229295 -0.03419925  0.04262431  0.01787596 -0.08484758
  0.04887678 -0.0504664  -0.01878674  0.00046867  0.01132252 -0.01971295
 -0.02109472  0.01797673  0.00819155 -0.0029198   0.00380794 -0.00232774
  0.02438789  0.02316296  0.00022886 -0.01671887  0.02626622  0.06080912
  0.01567925  0.01085247  0.04902368 -0.01575863  0.0804143  -0.01961805
  0.04047266 -0.09468596 -0.06591288  0.00678845 -0.01141438  0.01555718
  0.01061517 -0.01914072 -0.07251356 -0.01020163 -0.01773826 -0.06911764
 -0.01410793  0.01136851 -0.05967942  0.00968308 -0.01135633  0.07630897
  0.01566931  0.01668667 -0.04649669 -0.04623421  0.01267986 -0.02384517
  0.06250387  0.00870803  0.02352219  0.02776722 -0.06766813 -0.04914103
  0.02871088  0.0402616  -0.06333929  0.02953971  0.0356712   0.02148059
 -0.03829869 -0.07191611  0.00031206 -0.23724021  0.10377992 -0.02872021
  0.04441188 -0.01772385  0.00503825 -0.01966428 -0.07671531  0.0322194
  0.03569232 -0.0274316   0.033491   -0.02126963  0.00228425  0.02783309
  0.0110252   0.01285104 -0.04020981  0.00159809  0.00569644  0.00749118
 -0.03663905 -0.0273641  -0.03481447  0.04570457 -0.0613264  -0.05579361
 -0.03568013 -0.05757912 -0.01964564  0.10537192  0.01991254 -0.23034172
  0.02577833 -0.00772127  0.00247018 -0.04142473  0.01428352 -0.06699637
 -0.01055159 -0.00282744  0.0538788   0.02368198  0.02377288 -0.03277358
  0.03920718  0.00307108 -0.05687896  0.04820048  0.03696726  0.0110157
 -0.03109515  0.01848482 -0.09479371  0.0491099  -0.021566   -0.06104843
  0.01702824 -0.08156253 -0.07190847  0.09700389 -0.01429784 -0.03970926
  0.02038669  0.05935752 -0.04187482  0.02657081  0.08945996 -0.03799722
  0.02461291 -0.02605509 -0.06233663  0.01804635 -0.01783421 -0.04583948
  0.04419385  0.02172511 -0.01585858  0.02006212 -0.07538506  0.03268756
  0.015506   -0.05042879 -0.00509261 -0.00667143  0.03496018 -0.00132129
  0.00448079 -0.01487301  0.02960041 -0.02783187  0.05587982  0.08049795
 -0.0254826  -0.02364003 -0.04639492 -0.05233227 -0.02781267 -0.01041249
  0.00190702 -0.00377126 -0.02951425  0.09083881  0.02223902  0.01531031
 -0.01610169  0.0747937  -0.03033441  0.06144581  0.10713717 -0.05428004
  0.03272133 -0.04770003 -0.02756105 -0.05334713 -0.03550128  0.08314254
 -0.01996916 -0.01411308 -0.00204881 -0.03170719 -0.08364331 -0.00509663
 -0.0892911  -0.01837561 -0.03154976 -0.05373337 -0.02737207  0.0194218
  0.00151195  0.02779775  0.07034895  0.00176686  0.09333321 -0.03305516
 -0.02872086 -0.05489009  0.03383073  0.05652943  0.01468489 -0.01713363
  0.02295351  0.02968843  0.04376343  0.0231949  -0.00332519 -0.01968111
 -0.00031832  0.02289894  0.03041033 -0.00861475  0.00928742 -0.03981035
  0.04411198  0.00745245 -0.00519518  0.03495664  0.02614087 -0.00419489
 -0.01720672  0.01366337  0.08538387  0.02030778  0.02087771 -0.0480173
  0.03015785  0.04698307 -0.02644881 -0.02210315 -0.02551333  0.00574945
  0.02391769  0.01969932 -0.04403391 -0.0050654  -0.07376976  0.0251941
 -0.0325602   0.00766263  0.00455069  0.07310701 -0.05560024 -0.04398372
 -0.01087239  0.02921648  0.02274108 -0.02467595  0.01758958  0.18453638
 -0.09267632 -0.01384269 -0.01705925  0.01476767 -0.04114541 -0.01979959
 -0.05203665  0.01215836 -0.01607457  0.06227556 -0.02471347  0.08484634
  0.02035084  0.01577693 -0.03271595 -0.3182232  -0.04031724 -0.1147453
 -0.05878177  0.04495385 -0.04322866  0.02849701 -0.09521363  0.01486253
  0.01180752  0.01075791  0.01410832  0.03375013  0.05806322 -0.03914019
 -0.04103589  0.00873703  0.05120426  0.00528469 -0.04135979 -0.02757305
 -0.0669778  -0.0507758   0.03834682  0.03527409 -0.02384794  0.0574169
  0.03677482 -0.06025419  0.05917798 -0.04680705  0.04273957  0.0051242
  0.05808141 -0.00191473  0.00728531  0.01809977  0.05328935 -0.03167964
 -0.06718313  0.02811333  0.0220034   0.03396441  0.034638    0.00979545
  0.00039103  0.0220235   0.01572762 -0.01782368  0.05878346 -0.04555072
 -0.02346806  0.04174158  0.07445765 -0.01250274  0.01413212  0.04361488
  0.01714334 -0.00762622 -0.00897648  0.03808533  0.03210937  0.00266125
 -0.07338908 -0.03438803 -0.07352019 -0.04420791 -0.01985902  0.00561047
 -0.00508717  0.02523758  0.00800037  0.04249395  0.0132933  -0.00702462
  0.03546347  0.03352433  0.01689607 -0.05655351  0.08194404  0.03527687
 -0.02745127  0.00194224 -0.02591298 -0.02906713 -0.05394202  0.00063544
 -0.02034692 -0.04303518  0.05079146 -0.02232904  0.01147231 -0.034903
  0.01223234  0.03624441 -0.01121873  0.06435361  0.02499097  0.07007733
  0.0434062   0.03591991 -0.01633046  0.10392117  0.07456602  0.00538147
 -0.0227182   0.09076505 -0.02068618  0.05558537 -0.0136451  -0.01321183
 -0.05418062 -0.00172407  0.00818067  0.05151631 -0.00037038 -0.0063843
  0.03854876  0.06084701 -0.03914499  0.06477562 -0.05296099  0.04024998
  0.00302413  0.03061974 -0.01009763 -0.03526385  0.00049198  0.03254426
  0.01636919 -0.05683223 -0.05111432  0.02022477 -0.01226973 -0.1205639
 -0.00218736  0.04602748  0.00919885 -0.01756831 -0.0242746   0.03698894
 -0.01568145 -0.04502007  0.0155941  -0.02593307  0.06435608 -0.01268435
 -0.05269578  0.06487744 -0.0025922  -0.02819966 -0.0614607   0.04206697
 -0.00791258  0.0543965  -0.01068268  0.00579005 -0.08258599  0.01671163
  0.00097438 -0.02839516 -0.07337002 -0.01606756 -0.00686263 -0.03229005
 -0.03791342  0.07949884 -0.01347981 -0.04720963 -0.00778729 -0.04908907
  0.03054761  0.02496937  0.06692203  0.02753059 -0.0116412   0.03957853
 -0.00255172 -0.0285622   0.01193974 -0.01141666  0.08697192  0.00482859
  0.0122634   0.01800767 -0.01310454  0.07787012  0.01910247 -0.01573731
 -0.03026872  0.02115547  0.04050548 -0.02460679 -0.04584083 -0.03165377
 -0.00844778  0.01356202  0.10150617  0.01649137  0.00992922  0.03232093
  0.00733987 -0.09342155  0.03731516  0.03543925  0.03732708 -0.03318955
 -0.03648438  0.30469117 -0.0220605   0.04487576 -0.00095586 -0.00038489
  0.0187693  -0.04709497 -0.02600546 -0.03728654 -0.03056468 -0.02503239
  0.05209022 -0.04804997 -0.04706479 -0.05392402  0.05216108 -0.00271659
 -0.01385854  0.05296596  0.01725017 -0.05319231  0.0193103  -0.04132515
 -0.02597574 -0.00365769  0.02782565  0.035539   -0.01219313 -0.05568703
  0.05079437 -0.03507587  0.00866244 -0.04582352 -0.0716796  -0.3801557
  0.04141138 -0.01865985 -0.00032958  0.15127793  0.0661648   0.02353323
  0.04124271  0.01662739 -0.02765375  0.02633072 -0.02466716  0.06547041
  0.01532639 -0.03156079  0.00125764 -0.0110479   0.03525501 -0.05273596
  0.03626186  0.02105347 -0.04145983 -0.0253057   0.02667318 -0.00429113
  0.01949734 -0.09848816 -0.15905346 -0.03237807  0.00361055 -0.05560227
  0.04610404 -0.09871279  0.01987636  0.01436082 -0.00542197 -0.07941654
 -0.00758012  0.03555287 -0.03759248  0.00144882 -0.05876889 -0.04332322
 -0.04410789 -0.04088898 -0.30235144  0.08963428 -0.04045159 -0.03331012
  0.05270614  0.06225003  0.01530476 -0.04604986  0.03857874  0.07550984
 -0.00342595 -0.02530071 -0.00601736  0.02334216  0.00469786  0.03325573
 -0.01635845  0.00833611 -0.04479565 -0.05238895 -0.03790472 -0.01073934
  0.03121527 -0.0075279  -0.02988541 -0.01763249 -0.07900368  0.04709858
 -0.04229494  0.00813919 -0.02569047 -0.0298084  -0.02626441  0.02861808
  0.00465432  0.02971143  0.09364326 -0.03121197 -0.01055807 -0.05108861
 -0.02922243 -0.02818402  0.05753098 -0.00029262  0.03707648 -0.00016038
 -0.06724029 -0.02169323 -0.01443179 -0.01887755 -0.01588965  0.01113565
  0.02665631  0.04041271  0.0883221  -0.05516229  0.08807413  0.05967264
  0.07688257 -0.03793504  0.08433793 -0.02677945 -0.00175451 -0.02491445
 -0.04978592 -0.01517202 -0.03591401  0.02342087  0.03136685 -0.15664437
  0.01759322  0.01616857  0.02728915  0.05136629  0.02908663  0.01507987
  0.00774177 -0.00040674 -0.0620519   0.00467967  0.06001305 -0.07323964
 -0.01789819  0.0080173   0.00229189  0.05729618 -0.02770045  0.08015512
  0.03641988  0.0405899  -0.05915891  0.03467133 -0.00220615  0.04404858
  0.02578202 -0.02029387 -0.03795719 -0.00910578  0.01550684  0.06908022
 -0.01194078 -0.00453214 -0.02543586 -0.00460897 -0.02938556  0.02643885
 -0.00751733  0.1163772   0.09495539 -0.03536817 -0.00227231 -0.07518174
 -0.06942087  0.02905016 -0.03104796  0.04581114  0.00977294  0.05289187
  0.05499846 -0.08115586 -0.05103731 -0.04234653 -0.04645943  0.0515573
 -0.00435544  0.0598892  -0.06005532 -0.05730563  0.0217071  -0.03247488
  0.12279233 -0.03733464 -0.06061582 -0.08951937  0.0136157  -0.00262362
  0.02190607 -0.03835285 -0.27308786 -0.02528506 -0.05732388  0.01443753
 -0.05529022 -0.00660721  0.06241407  0.01740002  0.07062721  0.0983439
 -0.01848428  0.04061588 -0.00185302  0.05900651  0.04606496  0.05945847
  0.02822807 -0.00970876 -0.05740968 -0.00156323  0.02567436 -0.02444595
  0.11192262 -0.0669795   0.01013792  0.00819081 -0.00675698 -0.02610874
 -0.1621171   0.0026186   0.00545638 -0.04185756  0.04391396 -0.00670551
 -0.01814246  0.00980272  0.02140892 -0.06969028 -0.01482587  0.03121851
 -0.03591456 -0.0105018  -0.02477805 -0.04717241  0.03667556  0.01709661
  0.03085077 -0.00379112  0.02465795  0.02679411 -0.01662581  0.05828186
  0.01947268  2.1260657   0.06034587 -0.02344573  0.03160242 -0.05300129
  0.00915032 -0.04683878 -0.07476905 -0.06489744 -0.03315414  0.08280502
 -0.02598531  0.05004535 -0.05881128  0.00087959  0.04087829 -0.04834339
  0.02158998  0.00030925 -0.01173522 -0.04244484  0.11142789  0.01998605
  0.08076668 -0.03197618  0.05097428 -0.0342796   0.0021      0.0518871
  0.01449596 -0.02190231  0.04367252 -0.00202684 -0.0744848  -0.02018591
 -0.02201147 -0.0703382   0.00977901  0.0214673  -0.02394921  0.0478607
  0.04672932  0.04836118 -0.06483022  0.01382262  0.02037892 -0.06092058
  0.011556   -0.01203117  0.021308   -0.08507619  0.01798687 -0.06757221
  0.03254122 -0.06605513  0.21061651 -0.030895    0.01256166 -0.01461206
  0.01198239  0.0548167   0.0354316  -0.00609171  0.02115378  0.05092929
  0.0190684   0.05236645 -0.01575589  0.00637047 -0.03100269  0.02114678
  0.03570661  0.00180413  0.0719815   0.00714093 -0.08340269 -0.03345935
  0.01217632  0.04899723  0.03075124 -0.06681032  0.120515   -0.01989589
 -0.0188379  -0.10828669 -0.00792417 -0.02666589 -0.00286073 -0.04767879
  0.09299038  0.00412592  0.04419793  0.04691494  0.02166083  0.01192865
 -0.05532753  0.02990653 -0.0214315   0.00634228  0.04312405  0.15225886
 -0.1153423  -0.02931052 -0.03129765  0.01967594 -0.01664029  0.03237988
  0.03037003  0.01614082 -0.00658283  0.00853118 -0.03913058 -0.01971684
 -0.00035918  0.07017232  0.00346841 -0.08646519  0.00306329  0.03370075
 -0.01263896  0.07045978 -0.00276788 -0.02227229  0.04209924 -0.03964362
 -0.04515814  0.01125262  0.03363464 -0.06598677  0.16857348  0.00630443
  0.01027355  0.05065875 -0.02643816 -0.02045408 -0.05236459 -0.05265368
  0.082644    0.01748676 -0.00589774  0.06475835 -0.07189463 -0.02468809
 -0.04025897 -0.01823667 -0.06625155 -0.01511327 -0.00967616 -0.02260333
  0.00222958  0.07679871 -0.06933439  0.02546072 -0.00728636  0.08469082
 -0.07070654  0.02815769  0.03321496  0.05136192 -0.00824177  0.03636987
  0.00553603  0.06205334  0.02454928  0.06877585  0.0766096   0.0862793
 -0.07983993 -0.08789538  0.05911683 -0.01037774 -0.09954684 -0.03792526
  0.05186883  0.04783066 -0.05796248 -0.03004903 -0.04582771  0.01183766
  0.02622773 -0.01616605  0.0016625   0.01628928  0.04611217 -0.05257146
  0.01736545  0.03508069 -0.02994326  0.02817793  0.02755898  0.02359859
 -0.01666057 -0.00674255  0.03102154  0.0033791   0.00113235 -0.08300537
 -0.03665199 -0.03762186 -0.09023438  0.08986574 -0.03140564  0.00779927
 -0.23070465 -0.08524365  0.02348889  0.04177974 -0.03187978 -0.00818007
 -0.01967539 -0.01396288  0.00070925 -0.00159511  0.03006967  0.05462832
  0.02421012  0.02503004 -0.03879145 -0.03489209 -0.02065175 -0.04409601
  0.01001757 -0.0045734  -0.02809891 -0.03828458  0.03605133  0.02562355
 -0.01511379 -0.00828957  0.04717306 -0.04360535  0.0988052  -0.00628215
 -0.00995128  0.08194894  0.02856988  0.03610506 -0.02815601 -0.03789328
  0.02861047 -0.04220909 -0.07155921  0.01015653  0.01341636 -0.03817407
 -0.05421676 -0.02351089  0.01853107 -0.10829139 -0.01465518 -0.05992517
 -0.07915071 -0.04547732 -0.04636177  0.00127293
 """
# Example embedding

embedded_string = embedded_string.strip()
embedding_list = list(map(float, embedded_string.split()))
new_protein_embedding = np.array(embedding_list)

print(new_protein_embedding)

[ 0.09815001  0.01886502  0.00166107 ... -0.04547732 -0.04636177
  0.00127293]


In [9]:
embedded_string = """-0.04495116  0.06232567  0.04145993 -0.01739671 -0.0095748  -0.00405972
 -0.03625048  0.0292944  -0.03721101 -0.01931605 -0.00682326 -0.05557664
 -0.00822431  0.11374445  0.08269094 -0.01674263 -0.0499251   0.02229006
 -0.01509506 -0.03957492  0.04872627  0.06047805 -0.09853044 -0.04773638
 -0.02546417 -0.00184809  0.05559751 -0.03635503  0.03025961 -0.05485016
 -0.07728858 -0.02600227 -0.12485458  0.00370967  0.02670557 -0.0399172
 -0.01897555  0.01368715  0.02294967 -0.05016878  0.03182226  0.12839918
 -0.05018144  0.00760905  0.02281779 -0.04337019 -0.05174323 -0.06137564
  0.04340737 -0.03040709 -0.05190789  0.00086521 -0.02644611 -0.2024163
  0.00872994 -0.04025535  0.02371942 -0.10897617 -0.04452626 -0.04042935
  0.02268417 -0.04101783  0.0906423   0.06159677 -0.04456245  0.06602019
  0.01432717  0.03455278 -0.01940606  0.0296501   0.06724379 -0.02112198
 -0.05142916 -0.01513571  0.03853376  0.0195815  -0.03748599  0.0424209
  0.10732865 -0.09833238  0.0034089  -0.0132398  -0.02297756 -0.00543988
  0.02290174  0.04240106 -0.04112757  0.02385662  0.07426637 -0.02009819
 -0.00202006 -0.08225548  0.00425836 -0.04112084  0.04833879  0.00312731
  0.14016216 -0.03211767  0.01670619  0.07329237 -0.02549759  0.00085091
  0.00706903 -0.02336468  0.00709749 -0.04357066 -0.01312399 -0.02669225
 -0.06795901  0.04042667  0.00224384 -0.00546137  0.03954656  0.03120134
  0.00178192  0.01316135  0.05351833 -0.00818983  0.03655628 -0.01397834
 -0.00983801  0.05227265 -0.03827732 -0.02388719 -0.02206626  0.0745533
  0.02167502  0.08283681  0.04024818 -0.04493266 -0.04218574  0.03192499
  0.03399914 -0.00964851 -0.00495303 -0.00200729 -0.06143136  0.05277364
 -0.02817291  0.01108676 -0.05331589 -0.0465949   0.06383532 -0.24077955
 -0.00071331  0.31427497  0.01053919  0.00215717 -0.02172615  0.01280295
 -0.00349837 -0.00045556  0.01399084 -0.08592937 -0.18505484 -0.00025115
  0.01440134 -0.01053162 -0.06794883 -0.01047823 -0.05432656  0.02832458
  0.12068904  0.02168271  0.03240331  0.06637941 -0.02422621  0.01127377
 -0.05636403 -0.07740112  0.09259882  0.02340705  0.01221034 -0.008295
  0.05509941  0.03160807 -0.05483111 -0.11158493 -0.02787851  0.01752584
  0.07900251 -0.13890089 -0.0102349  -0.02951338 -0.00743975 -0.01435918
  0.01173567  0.05314089 -0.06263649 -0.0269112   0.04933879  0.05056262
  0.04261345  0.00515111  0.00239296 -0.03976152  0.12787019 -0.026884
 -0.02852867  0.05421409 -0.02179719 -0.00036377 -0.02195346 -0.04881079
  0.01397827  0.01072078 -0.04035947  0.0503063   0.0009261  -0.0082314
 -0.03449897  0.02408304 -0.01087689 -0.10278836 -0.05626764 -0.0353195
  0.04918493  0.06726877 -0.03741117 -0.02925922  0.40677887  0.01825779
  0.00074534 -0.03374087  0.02409606  0.05880013  0.05566834  0.10290236
 -0.0321378  -0.00925751 -0.04580183  0.00202815  0.01222539 -0.09945833
 -0.08337444  0.04986315 -0.5587389   0.01153304 -0.04501811  0.01780734
 -0.0106241  -0.05636416 -0.05788136  0.02987017  0.05357568 -0.05267713
 -0.02433331  0.00924327  0.0351757   0.00015833  0.04080371 -0.03585595
  0.1024435   0.01418948  0.0278757   0.01444808 -0.00165808 -0.01685755
  0.0116993   0.03563005  0.04984548 -0.04662901 -0.01956476  0.03122398
  0.02240916 -0.02714681 -0.05400995 -0.10233556 -0.04763442 -0.08713855
  0.03850087  0.00192855 -0.02628308  0.01181042 -0.01657085  0.14365716
 -0.04685692 -0.01169499  0.01574492  0.04356848 -0.01934551 -0.0501978
 -0.06127369 -0.03436711  0.00426993  0.07367702 -0.03335079  0.02041554
  0.04823796  0.03855519  0.03206612 -0.14318825  0.05058391  0.00954334
  0.02509848  0.02824876  0.05068162 -0.01136198 -0.00670346 -0.00393046
 -0.02293262 -0.03238789  0.01276991 -0.03464254 -0.05749955  0.01665218
 -0.08241336  0.00980952 -0.02929583  0.05377731  0.02544063 -0.029477
 -0.0804426   0.08444909 -0.01696323  0.02499544 -0.02125655 -0.02022973
 -0.01657345 -0.04121515 -0.0173261  -0.02159398  0.0412348   0.00156513
 -0.02312107 -0.01090487  0.01799539 -0.01765142  0.6445943   0.00225601
 -0.05999857  0.07022174  0.04047531 -0.00752473 -0.05189396 -0.01812403
  0.04483401 -0.04536961 -0.03478227 -0.01128032 -0.01529914 -0.03944868
 -0.0166048  -0.00827943 -0.03837964 -0.03266896  0.02034874 -0.00241676
  0.05721493 -0.0440659   0.00154793 -0.08941118 -0.00935099 -0.00093325
  0.00074338  0.05400068  0.01359979  0.03832552 -0.05947459 -0.02034608
  0.03793125  0.05255898 -0.00094896  0.10802566  0.00438618  0.04555462
  0.03822826 -0.04412139 -0.02166145 -0.06711327  0.17931265  0.03817412
  0.03846749 -0.01101353  0.02167565 -0.02077207 -0.06188156  0.01408458
 -0.04316617 -0.00903018 -0.05728706  0.0280791   0.00171202 -0.06904703
 -0.04399888 -0.01120094  0.06244943  0.06330743  0.04891705 -0.07706972
 -0.00879431 -0.0168453  -0.04113878 -0.07411767  0.12836903 -0.0080704
  0.07881922  0.16283268  0.05779547  0.0264277  -0.00407098 -0.0204345
 -0.05933094  0.05031644 -0.03937457 -0.02122477  0.02856239 -0.09531809
  0.08238446 -0.01723987  0.03040541 -0.00718353 -0.0110106  -0.05816207
 -0.02108433 -0.07258455  0.00317938  0.04949979  0.00391215 -0.06901671
 -0.01624383  0.04255044 -0.02267047  0.04843052 -0.03298894 -0.0240047
  0.10093464  0.07679237  0.06442202  0.01685235 -0.06316427  0.01370044
  0.07210757  0.04820974 -0.00294412 -0.3605373  -0.02297903 -0.06094417
 -0.03959969 -0.01933092 -0.04345425 -0.00318315 -0.01420603  0.01357574
 -0.02591713  0.04357208 -0.06674756 -0.02377303  0.01595056  0.01069971
  0.08793736  0.04256324 -0.01646304  0.03742559 -0.02115974 -0.01701537
  0.1020027   0.06179609 -0.05541958  0.07045699 -0.00886463  0.06706619
  0.03513712 -0.00205551  0.04800947 -0.02847756 -0.04670838  0.00963421
  0.00701888  0.05623834  0.06970296 -0.02911717  0.00117914  0.00122526
  0.0004947   0.00620334 -0.0611324  -0.01055485  0.05035567  0.01451229
 -0.02540919  0.02660201 -0.02124761 -0.04244692 -0.03199397  0.01821177
  0.08319843  0.00954235 -0.00155878 -0.04253246 -0.00599618  0.03252937
  0.00478651 -0.03319229  0.11442357 -0.04687029 -0.01561642 -0.12246247
 -0.04436666 -0.03713268 -0.01219325 -0.05167941 -0.01069836  0.02410801
 -0.08792859 -0.01342884  0.06758484 -0.02767366 -0.0612455  -0.0606018
  0.04970514  0.01155653 -0.01763739 -0.07026407  0.01432169 -0.07921579
  0.05745553 -0.01264559 -0.00294337 -0.01483666  0.01925666  0.00773989
 -0.1172653   0.04257853  0.00952909  0.02423007  0.01109452  0.09334436
  0.01781796  0.01979802 -0.08327871 -0.00791938  0.01610594 -0.67476094
 -0.01716688 -0.04738322 -0.07611442 -0.07740296  0.00651809  0.02604591
 -0.01161681  0.02873126 -0.12959854  0.12245122 -0.01066323  0.05539034
 -0.0552515  -0.22633885  0.03236442 -0.02968079  0.03716332  0.02378343
  0.03309823 -0.04465336  0.00591683  0.04715236 -0.00903678 -0.00170093
  0.01209393 -0.07175805 -0.12791605 -0.01943945 -0.07624143 -0.08677406
  0.07169601 -0.07084235  0.09464344 -0.00698133 -0.07050084  0.01800497
 -0.00267742 -0.0186658   0.01762536 -0.001228   -0.01120331  0.10677461
  0.0110872   0.00973667 -0.16671464  0.01126116  0.00534726 -0.07059269
  0.06512904 -0.01357846  0.10289438 -0.06744073  0.0163035   0.02726344
  0.03587724 -0.0266794  -0.03832069 -0.01592861 -0.03881838 -0.00056631
 -0.05181817 -0.04963953 -0.00533914 -0.00570244 -0.01658466 -0.04241342
  0.00260298  0.03113465 -0.02102519 -0.04317459  0.02790442 -0.01267812
  0.02327034 -0.05183913 -0.02197418  0.05847863 -0.04892249  0.02296067
  0.00985263 -0.04484355  0.04807893  0.03363149 -0.00937224  0.00826851
 -0.10066899  0.01838845  0.07082029  0.01861404 -0.03710093 -0.04267427
 -0.05884778  0.08970096 -0.03395641  0.03686053 -0.01046106  0.0061847
 -0.05563133 -0.01144747  0.05897491  0.00288965  0.06252394 -0.01357854
  0.01229443 -0.04884326 -0.07660539  0.02223803  0.05147088 -0.03627581
  0.00174141  0.04879839 -0.03139184  0.03246895 -0.04230387 -0.13883159
 -0.07081262  0.04184049  0.02069209  0.00762134 -0.04372168 -0.00173952
  0.00808727 -0.01211251 -0.01882537 -0.02203981  0.04030497 -0.02210851
  0.02729796 -0.02779632  0.03088992 -0.04017124  0.05364675  0.00002334
  0.04791349 -0.0385084   0.01116781  0.03488211  0.00372394  0.10088327
 -0.00788145 -0.00564244 -0.01500207  0.03488267 -0.00063101  0.03361892
  0.00107611 -0.03323337 -0.03537007 -0.03940285 -0.00252181  0.04254806
  0.28765386 -0.02331909  0.00272241 -0.07414488 -0.02738922 -0.01355242
 -0.01766692 -0.03027473 -0.00859038 -0.02346778  0.07378784  0.00427853
  0.06464639  0.0391904  -0.04624854 -0.00927592 -0.00235824  0.02790351
  0.00749101  0.02853451 -0.08563904  0.01933756  0.01784093 -0.03008785
 -0.06715936  0.04282143 -0.12982446 -0.02484983  0.01936677 -0.03285403
 -0.0374165  -0.05199207 -0.3948278  -0.01883007  0.00839073  0.13567322
  0.09542174 -0.01244438 -0.05525736  0.01970902  0.05306119  0.16113825
  0.00334786  0.02610135  0.00990316  0.02633893 -0.05781923  0.08806757
  0.00682049  0.01711859 -0.07128958 -0.01982285 -0.02957164 -0.03004171
  0.12477523  0.0220386   0.02951467  0.02263819  0.01412737 -0.01650686
 -0.0697121   0.04504024 -0.03021721 -0.00750116  0.05103381  0.03411842
  0.02043249  0.00201866 -0.03933944 -0.06628018 -0.0456057  -0.01080753
 -0.03509173  0.00505828 -0.00038028 -0.04579032  0.10934843 -0.05495004
  0.01389813 -0.02481259  0.03143828 -0.07953721 -0.04773956  0.02709134
 -0.06469338  5.093132    0.07483899  0.06122822 -0.01100241 -0.04032736
 -0.07354614  0.01658116  0.04444923  0.23109223 -0.00834939  0.03163497
  0.01039151  0.03704552  0.01549356 -0.01246001  0.16625163 -0.09116792
 -0.00856699 -0.02202794  0.02839422  0.7467921   0.03677225  0.00943818
  0.04756431  0.04550069 -0.00200503 -0.02124836 -0.06342018 -0.00423528
  0.01995643 -0.0300863  -0.00903302  0.00618625 -0.0803863  -0.03942713
  0.02646337 -0.06643457 -0.02545219 -0.00194413 -0.01315321  0.01417787
 -0.00891095  0.11037205 -0.00515509  0.03816735  0.00330577 -0.24887832
  0.01038992  0.02661564 -0.04869901 -0.01149748  0.15495802 -0.02138361
  0.06325568  0.27925766 -2.2106159  -0.03122961 -0.02711162 -0.00135283
  0.02797912 -0.0131967  -0.03214183 -0.00273364  0.02121201 -0.05162785
 -0.06179413 -0.00840565  0.01687298  0.09708195 -0.03095851  0.03647519
 -0.11939311 -0.01525318  0.04825342  0.03280887  0.04827624 -0.03051998
  0.00630021 -0.00681525  0.11229017 -0.00597221  0.02926474 -0.01134384
  0.00284769 -0.05847419 -0.00383537  0.05997385 -0.06268314 -0.0233082
 -0.00162159  0.04731041  0.03794578  0.00868151  0.02772032  0.01908323
  0.09511257 -0.03820029 -0.00212746  0.00166629  0.00915312  0.28504664
  0.03095022 -0.02572903  0.03166186 -0.0097948   0.03264353 -0.01637115
  0.09266806 -0.02548962  0.00649922  0.0766523  -0.0055309  -0.05111459
  0.0361017   0.07121237  0.03245537 -0.1093049   0.02274268  0.07726948
 -0.02971602  0.00221209  0.06871435 -0.00455847 -0.0023081  -0.01960561
  0.00285588  0.00975988 -0.00472865 -0.09550354  0.0532275   0.00868475
 -0.06508963 -0.05882017  0.04879608 -0.07715132  0.08621999 -0.06929958
 -0.08373168 -0.02660496  0.01226035  0.06642639  0.04018342 -0.00996975
  0.05043348 -0.00738536 -0.06806976  0.02327735  0.03910481  0.01760126
  0.03945133  0.01400709  0.01555583  0.04506589 -0.00725595 -0.0273512
  0.03613623  0.01867687  0.07276714  0.06875331  0.0033308  -0.04288596
 -0.07647162  0.02878577 -0.04700683  0.03307172 -0.02247938  0.02873796
  0.06857993  0.00121741 -0.04277914  0.04372777 -0.1089714   0.06534947
 -0.14235899 -0.00630856 -0.03035567 -0.04577509 -0.01339408 -0.00227524
 -0.018035    0.01174633 -0.09342505 -0.01274698 -0.05045252  0.00867791
 -0.01290386  0.01284585  0.02213853 -0.05972228  0.01557103 -0.01269315
  0.05118398 -0.02157119  0.02424419 -0.09166753 -0.06913248  0.02314406
  0.04033601 -0.08439591  0.02206449  0.01268362  0.03608853 -0.1164682
 -0.05158791 -0.01088091  0.04857967  0.0344845  -0.00019624 -0.03141312
  0.02760314 -0.01561047 -0.01942096  0.0469389   0.05355261 -0.00261727
  0.04270543  0.00482973  0.02927314  0.01468791  0.01577698  0.0561415
 -0.01949825 -0.0614046  -0.04263183  0.0096618  -0.03458893  0.00757446
 -0.02489335  0.0056967  -0.01683253  0.01394269 -0.00412665 -0.04418002
  0.04233527 -0.1892126  -0.01455481 -0.12242357 -0.01232459 -0.05479302
 -0.10466253  0.00404818  0.027872   -0.00212565  0.02262267 -0.01352884
  0.02868705 -0.02014975 -0.05489475 -0.04302471  0.02061548  0.00606677
 -0.02448449  0.02725065  0.01770933  0.06359288"""

embedded_string = embedded_string.strip()
embedding_list = list(map(float, embedded_string.split()))
new_protein_embedding = np.array(embedding_list)

print(new_protein_embedding)

[-0.04495116  0.06232567  0.04145993 ...  0.02725065  0.01770933
  0.06359288]


In [10]:
classifier = joblib.load("protein_function_classifier_with_hierarchy.pkl")
mlb = joblib.load("go_mlb_with_hierarchy.pkl")


new_protein_embedding = np.array(new_protein_embedding).reshape(1, -1)

predicted_labels = classifier.predict(new_protein_embedding)

predicted_go_terms = mlb.inverse_transform(predicted_labels)
print(f"Predicted GO terms: {predicted_go_terms}")


Predicted GO terms: [('GO:0003674',)]
